# 03 Build Macro Quarterly Features

Aggregate monthly predictors to quarterly, add lags, and merge with targets.


## Table of Contents
- [Aggregate monthly -> quarterly](#aggregate-monthly-quarterly)
- [Add lags](#add-lags)
- [Merge with GDP/labels](#merge-with-gdp-labels)
- [Save macro_quarterly.csv](#save-macro-quarterly-csv)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Data notebooks build the datasets used everywhere else. If these steps are wrong, every model result is suspect.
You will practice:
- API ingestion and caching,
- frequency alignment,
- label construction.


## Prerequisites (Quick Self-Check)
- Completed Part 00 (foundations) or equivalent time-series basics.
- FRED API key set (`FRED_API_KEY`) for real data (sample data works offline).

## What You Will Produce
- data/processed/macro_quarterly.csv

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.
- You can point to the concrete deliverable(s) listed below and explain how they were produced.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Merging mixed-frequency series without explicit resampling/aggregation.
- Forgetting to shift targets for forecasting tasks.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/01_data/03_build_macro_quarterly_features.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/01_data/03_build_macro_quarterly_features.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Build a quarterly modeling table by:
1) aggregating monthly predictors to quarterly features
2) adding lagged predictors (past-only)
3) merging with quarterly GDP growth + recession targets

The output is `data/processed/macro_quarterly.csv`.



## Primer: pandas time series essentials (indexing, resampling, lags)

Most “mysterious bugs” in time series work come from index and alignment mistakes. This primer gives you the minimum patterns to avoid them.

### 1) DatetimeIndex (the first thing to verify)

Most time-series operations assume a `DatetimeIndex`:

```python
import pandas as pd

df = df.copy()
df.index = pd.to_datetime(df.index)
df = df.sort_index()
assert isinstance(df.index, pd.DatetimeIndex)
```

**Expected output / sanity checks**
- `df.index.min(), df.index.max()` look reasonable
- `df.index.is_monotonic_increasing` is `True`

### 2) Resampling (frequency alignment)

Resampling converts one frequency to another. Choose the aggregation rule intentionally.

```python
# month-end last value (end-of-period)
df_me_last = df.resample("ME").last()

# month-end mean (average-of-period)
df_me_mean = df.resample("ME").mean()

# quarter-end mean
df_q_mean = df.resample("QE").mean()
```

**Interpretation matters**
- `.last()` treats end-of-period value as “the period’s value.”
- `.mean()` treats the period average as “the period’s value.”

### 3) Alignment and merging

When joining series, always check missingness after the join:

```python
merged = df1.join(df2, how="outer").sort_index()
print(merged.isna().sum().sort_values(ascending=False).head(10))
```

### 4) Lags and rolling windows (watch for leakage!)

```python
# lag 1 period (past-only)
df["x_lag1"] = df["x"].shift(1)

# rolling mean using past values ending at t
df["x_roll12"] = df["x"].rolling(12).mean()
```

**Leakage pitfalls**
- `shift(-1)` uses the future.
- `rolling(..., center=True)` uses the future.

### 5) A quick workflow you should repeat

1) Set and verify DatetimeIndex.
2) Resample intentionally (mean vs last).
3) Join and inspect missingness.
4) Add lags/rolls (past-only).
5) `dropna()` to build a clean modeling table.


<a id="aggregate-monthly-quarterly"></a>
## Aggregate monthly -> quarterly

### Goal
Convert the month-end panel into a quarterly feature table.

You will try two aggregation rules:
- quarter-end value (`last`)
- quarter-average value (`mean`)

Then you will choose one (or keep both with suffixes).



### Your Turn (1): Load inputs


In [ ]:
import pandas as pd

# TODO: Load the processed monthly panel (or fallback to sample)
panel_path = PROCESSED_DIR / 'panel_monthly.csv'
if panel_path.exists():
    panel_m = pd.read_csv(panel_path, index_col=0, parse_dates=True)
else:
    panel_m = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)

# TODO: Load quarterly GDP/label table from the previous notebook
gdp_path = PROCESSED_DIR / 'gdp_quarterly.csv'
if gdp_path.exists():
    gdp_q = pd.read_csv(gdp_path, index_col=0, parse_dates=True)
else:
    gdp_q = pd.read_csv(SAMPLE_DIR / 'gdp_quarterly_sample.csv', index_col=0, parse_dates=True)

panel_m.head(), gdp_q.head()



### Your Turn (2): Aggregate (mean vs last)


In [ ]:
from src import macro

# TODO: Build quarterly versions of the monthly predictors.
# Hint: macro.monthly_to_quarterly(panel_m, how='mean'|'last')
panel_q_last = ...
panel_q_mean = ...

# TODO: Compare them (e.g., correlation of each column)
...



### Your Turn (3): Choose a quarterly feature table


In [ ]:
# TODO: Choose which aggregation to use for modeling.
# Option A: use quarter-end values
# Option B: use quarter averages
# Option C: keep both by adding suffixes

Xq = ...
Xq.head()



### Checkpoint (quarterly index alignment)


In [ ]:
# TODO: Confirm both Xq and gdp_q use quarter-end timestamps.
assert isinstance(Xq.index, pd.DatetimeIndex)
assert isinstance(gdp_q.index, pd.DatetimeIndex)
assert Xq.index.is_monotonic_increasing
assert gdp_q.index.is_monotonic_increasing
...



<a id="add-lags"></a>
## Add lags

### Goal
Add lagged quarterly predictors so the model only uses information available *before* the target period.

Typical lags to try:
- 1 quarter
- 2 quarters
- 4 quarters (one year)



### Your Turn (1): Add lag features


In [ ]:
from src import features

# TODO: Add lag features for all columns in Xq
# Hint: features.add_lag_features(Xq, columns=Xq.columns, lags=[...])
Xq_lagged = ...

# TODO: Drop rows with NaNs created by lags
Xq_lagged = Xq_lagged.dropna().copy()
Xq_lagged.head()



### Checkpoint (no future lags)


In [ ]:
# TODO: Confirm you used ONLY positive lags.
# features.add_lag_features will raise if lags <= 0.
...



<a id="merge-with-gdp-labels"></a>
## Merge with GDP/labels

### Goal
Join lagged predictors with GDP growth and the next-quarter recession target.

Key idea:
- predictors at time t
- target at time t (which is recession at t+1)



### Your Turn (1): Merge and build the final table


In [ ]:
# TODO: Join on the quarterly index.
# Keep at least:
# - gdp growth columns
# - recession label
# - target_recession_next_q
# - lagged predictors

df_q = ...

# Drop rows with missing target or predictors
df_q = df_q.dropna().copy()
df_q.head()



### Checkpoint (target alignment)


In [ ]:
# TODO: Confirm the target is 0/1 and shifted correctly.
assert set(df_q['target_recession_next_q'].unique()).issubset({0, 1})
...



<a id="save-macro-quarterly-csv"></a>
## Save macro_quarterly.csv

### Goal
Write the final modeling table to `data/processed/macro_quarterly.csv`.



### Your Turn (1): Save + reload


In [ ]:
out_path = PROCESSED_DIR / 'macro_quarterly.csv'
out_path.parent.mkdir(parents=True, exist_ok=True)

# TODO: Save df_q
...

# Reload for sanity
df_reload = pd.read_csv(out_path, index_col=0, parse_dates=True)
assert df_reload.shape == df_q.shape
df_reload.tail()



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# Expected file: data/processed/macro_quarterly.csv
# TODO: After saving your processed dataset, load it and run checks.
# df = pd.read_csv(PROCESSED_DIR / 'your_file.csv', index_col=0, parse_dates=True)
# assert df.index.is_monotonic_increasing
# assert df.shape[0] > 20
# print(df.dtypes)
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Aggregate monthly -> quarterly</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_build_macro_quarterly_features — Aggregate monthly -> quarterly
import pandas as pd
from src import macro

panel_monthly = pd.read_csv(SAMPLE_DIR / 'panel_monthly_sample.csv', index_col=0, parse_dates=True)
q_mean = macro.monthly_to_quarterly(panel_monthly, how='mean')
q_last = macro.monthly_to_quarterly(panel_monthly, how='last')
q_mean.head(), q_last.head()
```

</details>

<details><summary>Solution: Add lags</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_build_macro_quarterly_features — Add lags
q = q_mean.copy()
for col in q.columns:
    for lag in [1, 2, 4]:
        q[f'{col}_lag{lag}'] = q[col].shift(lag)
q = q.dropna()
```

</details>

<details><summary>Solution: Merge with GDP/labels</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_build_macro_quarterly_features — Merge with GDP/labels
gdp = pd.read_csv(SAMPLE_DIR / 'gdp_quarterly_sample.csv', index_col=0, parse_dates=True)
df = q.join(gdp, how='inner').dropna()
df.head()
```

</details>

<details><summary>Solution: Save macro_quarterly.csv</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 03_build_macro_quarterly_features — Save macro_quarterly.csv
from src import data as data_utils
data_utils.save_csv(df, PROCESSED_DIR / 'macro_quarterly.csv')
print('saved', PROCESSED_DIR / 'macro_quarterly.csv')
```

</details>

